In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


Eu tenho um arquivo secret.py com uma informação sigilosa. Neste caso é uma chave.
Eu vou chamar o conteudo dessa chave usando o código abaixo.


In [3]:
from secret import IEX_CLOUD_API_TOKEN

## Fazer a primeira chamada a API

A api que eu estou usando

A documentação da API que eu estou usando se encontra aqui: https://iexcloud.io/docs/api/


Eu vou buscar na API duas informações importantes:
O market capitalization para cada ação e o preço de cada ação.

Abaixo eu vou usar fstream para passar o simbolo para o endereço.

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)
# print(data.status_code)
# Para eu converter o dado recebido na request em json eu só preciso
# Usar o comando abaixo.
data = requests.get(api_url).json()
print(data)


{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'E ESLSRDAAQA(T)COSGML T NB/LGAKEN', 'calculationPrice': 'iexlasttrade', 'open': None, 'openTime': None, 'openSource': 'afcfliio', 'close': None, 'closeTime': None, 'closeSource': 'ilcofafi', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 140.22, 'latestSource': 'IEX Last Trade', 'latestTime': 'February 9, 2021', 'latestUpdate': 1633539934381, 'latestVolume': None, 'iexRealtimePrice': 139.42, 'iexRealtimeSize': 104, 'iexLastUpdated': 1620531814861, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 138.65, 'previousVolume': 73819377, 'change': -0.87, 'changePercent': -0.00653, 'volume': None, 'iexMarketPercent': 0.013293093508357718, 'iexVolume': 1050518, 'avgTotalVolume': 1093

### Um pouco sobre fstrem

Eu uso fstream para modificar alguma tag.

Por exemplo no código abaixo, eu vou trocar a tag {adjective} pela variável adjective acima.

In [5]:
adjective = "superb"
string = f"freeCodeCamp is {adjective}"
print(string)

freeCodeCamp is superb


Parei em 37:45

## Fazer parser do resultado que eu peguei na request da API

In [6]:
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap)

2370374095848


## Adicionar as informações das ações em um Pandas DataFrame

Para eu passar as informações para um Pandas DataFrame eu preciso antes formatar o DataFrame

In [7]:
# Criar as colunas do dataframe
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


Agora que eu criei o DataFrame, eu vou fazer um append das minhas informações no DataFrame que eu criei logo acima.

Para eu poder adicionar uma informação no meu dataframe eu preciso converter a minha informação em um Panda series. então, se eu tiver um vetor em python que eu quero fazer um append em um dataframe, antes eu preciso converter o vetor em um `pandas series`. 

In [8]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,140.22,2370374095848,N/A


## Adicionando todas as informações das ações no DataFrame usando loop

Eu tenho uma lista com todos as açções. então eu vou usar a minha lista para passar o nome de cada ação para o endpoint que eu pego as informaçoes da ação na minha API.

O código abaixo vai executar muito demorado. Pois ele vai fazer uma request por vez.
Depois eu vou fazer uma forma que eu faça todos os request em batch de uma única só vez.
Pra não demorar tanto eu vou fazer o append apenas das 5 primeiras ações da minha lista de 500 ações.

In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
    [
        stock,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ],
    index = my_columns),
    ignore_index = True)

In [10]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,124.88,37843269854,N/A
1,AAL,17.49,10881729662,N/A
2,AAP,166.14,11243781975,N/A
3,AAPL,137.48,2303593149572,N/A
4,ABBV,110.10,187921445426,N/A


## Utilizando Batch API Calls para melhorar a performace 

Primeiro eu preciso fazer um split da minha lista em sublistas.

In [11]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

O marketCap é o valor da empresa ou algo do tipo. Por isso que é um valor tão grande.

In [12]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
# symbol_strings vai ser uma lista com todos os nomes de todas as ações
# que eu estou trabalhando. 
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

# Criação de um dataFrame vazio, apenas com a primeira linha que são
#as colunas
final_dataframe = pd.DataFrame(columns = my_columns)

# Fazer uma request em batches. Acontece que aqui eu vou ter uma lista com
# diversos simbolos e eu vou fazer uma request de todos os simbolos 
# de uma única vez
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],
                index = my_columns),
                ignore_index=True
        )
print(final_dataframe)


    Ticker  Stock Price Market Capitalization Number of Shares to Buy
0        A      125.200           38932889493                     N/A
1      AAL       17.930           10646398532                     N/A
2      AAP      167.140           11441937932                     N/A
3     AAPL      139.990         2308937579760                     N/A
4     ABBV      110.200          188123459864                     N/A
..     ...          ...                   ...                     ...
500    YUM      104.880           31644898678                     N/A
501    ZBH      165.620           33311671698                     N/A
502   ZBRA      433.663           23264947156                     N/A
503   ZION       50.500            8329391275                     N/A
504    ZTS      167.430           76393925365                     N/A

[505 rows x 4 columns]


## Calculating the Number of Shares to Buy

Aqui eu vou calcular a quantidade de shares (ações) de cada ação/empresa (stock) que eu quero comprar.

In [13]:
portfolio_size = input('Enter the value of your portfolio: ')

try:
    val = float(portfolio_size)
except ValueError:
    print("O que você colocou não é um número!\nPor favor tente novamente:")
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)

Enter the value of your portfolio: 100000


In [49]:
# position_size é o valor que vai ser dedicado para cada ação.
# Colocar um valor a partir de 1000000 no portfolio_size. Pois essas
# Ações são muito caras e são 505 ações. Menos que isso não dá pra aplicar
# em todas as ações desse conjunto.
position_size = val/len(final_dataframe.index)
print(position_size)
#final_dataframe.index - quantidade de linhas de um dataframe
# final_dataframe.loc[i, 'Number of Shares to Buy'] - maneira de você
# ter acesso a um elemento do seu dataframe
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

final_dataframe

198.01980198019803


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,125.200,38932889493,1
1,AAL,17.930,10646398532,11
2,AAP,167.140,11441937932,1
3,AAPL,139.990,2308937579760,1
4,ABBV,110.200,188123459864,1
...,...,...,...,...
500,YUM,104.880,31644898678,1
501,ZBH,165.620,33311671698,1
502,ZBRA,433.663,23264947156,0
503,ZION,50.500,8329391275,3


Terminei em 1:22:00

## Formatando o Excel Output
Para formatar o excel, eu vou usar a biblioteca de python XlsxWriter.

In [43]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)
# Perceba que embora eu tenha criado o xls eu não salvei. Então para eu ver 
# o arquivo real eu preciso usar a função save - ex: writer.save()

### Criando formatação do xls
Perceba que primeiro você cria op xls e depois que você criou o xls e adicionou lá as informações, é hora de fazer a formatação das colunas.
A conclusão é que a formatação é feita depois que o xls já foi criado.

Essa parte de formatação é a parte mais complicada do processo.

In [44]:
background_color = '#0a0a23'
font_color = '#ffffff'

# Vou criar aqui uma formatação para as celulas
# 'border': 1 - significa que a borda vai ser sólida de espessura
# igual a 1.
string_format = writer.book.add_format(
{
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
}
)

string_format = writer.book.add_format(
{
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
}
)

dollar_format = writer.book.add_format(
{
    'num_format': '$0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
}
)

interger_format = writer.book.add_format(
{
    'num_format': '0',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
}
)



### Aplicando a formatação que eu criei no xlsx file
Perceba que acima eu criei apenas as formatações que eu desejo para os meus arquivos. Agora que eu criei as formatações eu preciso adicionar no arquivo.

Essa parte é bastante complexa. Então veja que os detalhes são mais complicados que a própria adição dos arquivos.

In [45]:
"""writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
writer.save()"""

"writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)\nwriter.sheets['Recommended Trades'].set_column('B:B', 18, string_format)\nwriter.sheets['Recommended Trades'].set_column('C:C', 18, string_format)\nwriter.sheets['Recommended Trades'].set_column('D:D', 18, string_format)\nwriter.save()"

Só com o código acima eu vou ter as minhas colunas formatadas, PORÉM eu não vou ter formatado o header do arquivo. 
Abaixo eu vou colocar como formatar o header do arquivo.

Aqui eu vou fazer a formatação de maniera mais simples que lá em cima.
Então eu vou tá fazendo a mesma formatação, porém de maneira mais simplificada usando for.

In [46]:
# Criar a formatação do header
""""writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', interger_format)"""

'"writer.sheets[\'Recommended Trades\'].write(\'A1\', \'Ticker\', string_format)\nwriter.sheets[\'Recommended Trades\'].write(\'B1\', \'Stock Price\', dollar_format)\nwriter.sheets[\'Recommended Trades\'].write(\'C1\', \'Market Capitalization\', dollar_format)\nwriter.sheets[\'Recommended Trades\'].write(\'D1\', \'Number of Shares to Buy\', interger_format)'

In [47]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', interger_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    # Adição da formatação para o header
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])



## Salvar o xls na pasta
O código abaixo é o que realmente faz o xls ser salvo.

In [48]:
writer.save()